In [1]:
import requests
import base64
import os
import json
from flask import Flask
import urllib.parse 
import pandas as pd
from IPython.display import Image, display, HTML

In [2]:
def extract_spotify_url(external_urls):
    return external_urls['spotify']
    
def make_clickable(link):
    return f'<a href="{link}" target="_blank">{link}</a>'

https://developer.spotify.com/documentation/web-api/tutorials/getting-started

In [3]:
def get_spotify_access_token(client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    headers = {
        'Authorization': 'Basic ' + base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode()
    }
    data = {
        'grant_type': 'client_credentials'
    }
    
    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()  # Solleva un'eccezione se la richiesta HTTP non ha successo
        token_data = response.json()
        
        if 'access_token' in token_data:
            return token_data['access_token']
        else:
            raise Exception(f"Errore nell'ottenere il token di accesso: {token_data.get('error', 'Errore sconosciuto')}")
            
    except requests.exceptions.RequestException as e:
        raise Exception(f"Errore nella richiesta HTTP: {e}")

In [4]:
def search_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Solleva un'eccezione se la richiesta HTTP non ha successo
        data = response.json()
        artists = data.get("artists", {}).get("items", [])
        
        if artists:
            return artists[0]
        else:
            print(f"Nessun artista trovato con il nome '{artist_name}'.")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta HTTP: {e}")
        return None


In [5]:
def get_albums(token, artist_id):
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums'
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "market": "IT"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Solleva un'eccezione se la richiesta HTTP non ha successo
        data = response.json()
        
        return data.get("items", [])
        
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta HTTP: {e}")
        return []

In [6]:
def get_album_tracks(token, album_id):
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    
    headers = {
        "Authorization": f"Bearer {token}"
    }
    params = {
        "market": "IT"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # Solleva un'eccezione se la richiesta HTTP non ha successo
        data = response.json()
        
        return data.get("items", [])
        
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta HTTP: {e}")
        return []

In [7]:
def top_tracks_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=IT"
    headers = {
        "Authorization": f"Bearer {token}"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  
        data = response.json()["tracks"]
        return data
        
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta HTTP: {e}")
        return None

## Testing delle funzioni realizzate

In [8]:
client_id = 'cd0f7e9279034469a5c4560808cb31a8' 
client_secret = 'da3fe74efc5c4a84b71399c157366e09'
access_token = get_spotify_access_token(client_id, client_secret) # ottengo il token di accesso

print(f"Token di accesso: {access_token}")

Token di accesso: BQBU84eTK1hH772Qqh6I-kwiN0E3o9kW8OtInP_GKToaiZNpD55F1BEgvkWOCinyhJ7D2PAYqjSAqGYO3W_V8qCb7dF0ePOUIlKMzQlwdnG75WYYHds


In [9]:
artist_name = "Emis killa" 
artist = search_artist(access_token, artist_name) # ottengo l'artista

artist_data = {
    "name": artist["name"],
    "id": artist["id"] ,
    "external_urls": artist["external_urls"]["spotify"],
    "genres": artist["genres"][0],
    "followers": artist["followers"]["total"],
    "images_url": artist["images"][0]["url"]
}
pd.set_option('display.max_colwidth', 1)
artist_data_frame = pd.DataFrame([artist_data])

print(f"Artista trovato: {artist_name} (ID: {artist_data['id']})\n") 

# Visualizza l'immagine centrata
image_html = f'<div style="display: flex; justify-content: center;">' \
             f'<img src="{artist_data_frame["images_url"][0]}" alt="Artista" width="{250}" style="display: block;"/>' \
             f'</div>'
artist_data_frame.drop("images_url", axis=1)
display(HTML(image_html))

#artist_data_frame drop rimuove la collone Artist image URL e visualizza il dataframe
display(artist_data_frame)

Artista trovato: Emis killa (ID: 6FtwCmLY6L1sqvjaQ2lV6G)



,name,id,external_urls,genres,followers,images_url
0,Emis Killa,6FtwCmLY6L1sqvjaQ2lV6G,https://open.spotify.com/artist/6FtwCmLY6L1sqvjaQ2lV6G,italian hip hop,1545437,https://i.scdn.co/image/ab6761610000e5eb51507026f810de5f84d9e8a2


In [10]:
artist_albums = get_albums(access_token, artist_data['id']) # ottengo gli album dell'artista
album_data_frame = pd.DataFrame(artist_albums, columns=['name','id','total_tracks','release_date','album_type','uri'])

print(f"Gli album dell'artista {artist_name} sono:\n")
album_data_frame

Gli album dell'artista Emis killa sono:



,name,id,total_tracks,release_date,album_type,uri
0,Effetto notte,39wuRno08Rl0Jsse2kCHga,14,2023-05-19,album,spotify:album:39wuRno08Rl0Jsse2kCHga
1,L'erba cattiva (10 Years Anniversary Edition),53qzcXW1K4TdnJZUp0Ftpu,15,2022-12-09,album,spotify:album:53qzcXW1K4TdnJZUp0Ftpu
2,"Keta Music, Vol. 3",5XmLyPktaR9fYxYqtQukqa,11,2021-07-23,album,spotify:album:5XmLyPktaR9fYxYqtQukqa
3,17 - Dark Edition,6nKQCH9bP1APHf83iDCjO2,24,2021-02-26,album,spotify:album:6nKQCH9bP1APHf83iDCjO2
4,17,11T8pkk7VAT7LeoReyY7B7,17,2020-09-18,album,spotify:album:11T8pkk7VAT7LeoReyY7B7
5,Supereroe Bat Edition,3YdgMr7sbDBeuPotzWJz60,17,2019-06-21,album,spotify:album:3YdgMr7sbDBeuPotzWJz60
6,Supereroe,67g3eUjlYFB7EkhS1pjKe4,13,2018-10-12,album,spotify:album:67g3eUjlYFB7EkhS1pjKe4
7,Terza Stagione,2goDKoHetJSDYCxZq9h17h,17,2016-10-14,album,spotify:album:2goDKoHetJSDYCxZq9h17h
8,Keta Music - Volume 2,2P8Ga6LVlc6lwuKwxsrbsd,13,2015-06-18,album,spotify:album:2P8Ga6LVlc6lwuKwxsrbsd
9,Mercurio - 5 Stars Edition,5ZPw8Dc0rcfLBlJj9zJ9pK,19,2014-06-10,album,spotify:album:5ZPw8Dc0rcfLBlJj9zJ9pK


In [11]:
album_name = "Effetto notte"  
album_id = album_data_frame.loc[album_data_frame['name'] == album_name, 'id'].values[0]
album_tracks = get_album_tracks(access_token, album_id)
tracks_data_frame = pd.DataFrame(album_tracks, columns=['name','id','duration_ms','external_urls'])

tracks_data_frame['duration_ms'] = tracks_data_frame['duration_ms'] / 60000
tracks_data_frame['duration_ms'] = tracks_data_frame['duration_ms'].round(2)

tracks_data_frame['external_urls'] = tracks_data_frame['external_urls'].apply(extract_spotify_url)
tracks_data_frame['external_urls'] = tracks_data_frame['external_urls'].apply(make_clickable)
tracks_data_frame = tracks_data_frame.rename(columns={'external_urls':'spotify_url'})

print(f"Le tracce dell'album {album_name} sono:\n")
display(HTML(tracks_data_frame.to_html(escape=False)))

Le tracce dell'album Effetto notte sono:



,name,id,duration_ms,spotify_url
0,PACINO - the godfather,0DJJkaSRugtaqVjWOItGn0,2.04,https://open.spotify.com/track/0DJJkaSRugtaqVjWOItGn0
1,ON FIRE - paid in full,2pGbOFiWVwZaTq3iKi5sGb,2.93,https://open.spotify.com/track/2pGbOFiWVwZaTq3iKi5sGb
2,COL CUORE IN GOLA - lords of dogtown,2ypcWaliFPJ9c3JeWMqi1w,3.14,https://open.spotify.com/track/2ypcWaliFPJ9c3JeWMqi1w
3,VISCERALE - closer,7bgJm5Z7EpvXRMdysiUjCj,3.07,https://open.spotify.com/track/7bgJm5Z7EpvXRMdysiUjCj
4,MASERATI - the wolf of wall street,3vZ3ZCdyr7jxBDKzgFtKBN,2.27,https://open.spotify.com/track/3vZ3ZCdyr7jxBDKzgFtKBN
5,LONTANO - carlito's way,2t1dculU2L9HxEIzB00Xpk,2.78,https://open.spotify.com/track/2t1dculU2L9HxEIzB00Xpk
6,ALBICOCCA - lolita,1Bkq1PP6QZrZcpAyEnIKDy,2.71,https://open.spotify.com/track/1Bkq1PP6QZrZcpAyEnIKDy
7,BIG BANG - upside down,6IMdD50UNZOZxGro2bgsnN,3.30,https://open.spotify.com/track/6IMdD50UNZOZxGro2bgsnN
8,TOXIC - trainspotting,3mrtP5imQ3Mv3OQ0byI4ix,3.00,https://open.spotify.com/track/3mrtP5imQ3Mv3OQ0byI4ix
9,SONNY - city of god,3knzVyYsKLWzlzdBiZPHWB,3.18,https://open.spotify.com/track/3knzVyYsKLWzlzdBiZPHWB


In [12]:
top_tracks = top_tracks_artist(access_token, artist_data['id'])
top_tracks_data_frame = pd.DataFrame(top_tracks, columns=['name','popularity','external_urls'])

top_tracks_data_frame['external_urls'] = top_tracks_data_frame['external_urls'].apply(extract_spotify_url)
top_tracks_data_frame['external_urls'] = top_tracks_data_frame['external_urls'].apply(make_clickable)
top_tracks_data_frame = top_tracks_data_frame.rename(columns={'external_urls':'spotify_url'})

print(f"Top Tracks dell'artista {artist_name} :\n")
display(HTML(top_tracks_data_frame.to_html(escape=False)))

Top Tracks dell'artista Emis killa :



,name,popularity,spotify_url
0,ON FIRE - paid in full,67,https://open.spotify.com/track/2pGbOFiWVwZaTq3iKi5sGb
1,SENZ'ANIMA - nikita,64,https://open.spotify.com/track/39ZJEJ8KLq8yQQ1OEAdJ1W
2,Fuoco e benzina,63,https://open.spotify.com/track/4Fs1sEg9Mwn6vykFD89xvc
3,Mon Fre - feat. Emis Killa,62,https://open.spotify.com/track/4dDMgEj5GpBTI1lvVlYPu7
4,Rollercoaster,56,https://open.spotify.com/track/4WJj8NsyCm3YAADCOe0hrf
5,Parole di ghiaccio,60,https://open.spotify.com/track/1mItEeAlTepKfWUOpu8h6b
6,"PAROLA - feat. Lazza, Emis Killa",52,https://open.spotify.com/track/4XxQOwmNM0DF3bKafSJBNw
7,L'ultima volta - feat. Massimo Pericolo,58,https://open.spotify.com/track/3rOw9WmYFCRLHTRcbOjhUl
8,COL CUORE IN GOLA - lords of dogtown,58,https://open.spotify.com/track/2ypcWaliFPJ9c3JeWMqi1w
9,No Cap (feat. Geolier),58,https://open.spotify.com/track/3uqFFmxrCv4nzkKFSNbXHv


from flask import Flask, redirect, request, url_for
import random
import string
import urllib.parse

app = Flask(__name__)

# Definisci il tuo client_id e redirect_uri
redirect_uri = 'http://localhost:5000/callback'

# Funzione per generare una stringa casuale
def generate_random_string(length):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for _ in range(length))

@app.route('/login')
def login():
    state = generate_random_string(16)
    scope = 'user-read-private user-read-email'
    params = {
        'response_type': 'code',
        'client_id': client_id,
        'scope': scope,
        'redirect_uri': redirect_uri,
        'show_dialog': True,
        'state': state
    }
    query_string = urllib.parse.urlencode(params)
    return redirect('https://accounts.spotify.com/authorize?' + query_string)

if __name__ == '__main__':
    app.run(debug=True, port=5000)